In [1]:
source activate qiime2-2021.11

(qiime2-2021.11) 

: 1

# Importing and Trimming Data #

In [3]:
qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]'\
    --input-path DSS2_manifest.csv \
    --output-path paired-end-demux.qza \
    --input-format PairedEndFastqManifestPhred33

#Importing data from fastq files
#usually we would need to demultiplex but since we are using the manifest, this is already done
#Phred33 refers to the cutoff of sequence quality, which we are using as 33

Imported DSS2_manifest.csv as PairedEndFastqManifestPhred33 to paired-end-demux.qza
(qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) 

: 1

In [3]:
qiime demux summarize \
--i-data paired-end-demux.qza \
--o-visualization paired-end-demux.qzv

Saved Visualization to: paired-end-demux.qzv
(qiime2-2021.8) 

: 1

In [7]:
qiime tools view paired-end-demux.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 

# DADA2 #

In [3]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs paired-end-demux.qza \
--o-table table.qza \
--o-representative-sequences rep-seqs.qza \
--p-trim-left-f 24 \
--p-trim-left-r 18 \
--p-trunc-len-f 263 \
--p-trunc-len-r 238 \
--p-min-fold-parent-over-abundance 5 \
--p-n-threads 0 \
--o-denoising-stats stats-dada2.qza \
--verbose

#using our demultiplexed sequences and info from our interactive quality plot, see where quality drops off 
#for both forward and reverse, at the beginning and end, and cut there
#originally had a high proportion of chimeras, even after trimming primers off
#changed chimera stringency in dada2 using the p-min fold over abundance to 5 from default 2

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmp4z_04a88/forward /tmp/tmp4z_04a88/reverse /tmp/tmp4z_04a88/output.tsv.biom /tmp/tmp4z_04a88/track.tsv /tmp/tmp4z_04a88/filt_f /tmp/tmp4z_04a88/filt_r 300 271 0 0 2.0 2.0 2 12 independent consensus 5.0 0 1000000

R version 4.0.5 (2021-03-31) 
Loading required package: Rcpp
DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering Error in names(answer) <- names1 : 
  'names' attribute [171] must be the same length as the vector [149]
Execution halted
Traceback (most recent call last):
  File "/home/etb/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/q2_dada2/_denoise.py", line 266, in denoise_paired
    run_commands([cmd])
  File "/home/etb/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/q2_dada2/_d

: 1

In [8]:
qiime metadata tabulate \
--m-input-file stats-dada2.qza \
--o-visualization stats-dada2.qzv

#An important check here to see how many reads passed through each dada2 step.
#Unless quality is poor, should have over 75% of reads at the end.
#otherwise-- issue with sequencing run, primer removal etc.

Saved Visualization to: stats-dada2.qzv
(qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) 

: 1

In [5]:
qiime tools view stats-dada2.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 

# Feature Table

In [7]:
qiime feature-table summarize \
    --i-table table.qza \
    --o-visualization table.qzv \
    --m-sample-metadata-file DSS2_metadata.tsv

#creating a visual summary of the data
#inputting the metadata file here - associating a time point, treatment group, etc to each sample

Saved Visualization to: table.qzv
(qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) 

: 1

In [8]:
qiime tools view table.qzv 

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 

Here I am just prematurely filtering my table to get rid of the positive & negative sequencing controls, as well as the sheath fluid samples

Note that nc = no sequencing controls, ns = no sheath fluid, ncns = no controls and no sheath fluid, nnn = no controls, no sheath fluid, no contamination

### No sequencing controls

In [9]:
qiime feature-table filter-samples \
--i-table table.qza \
--m-metadata-file DSS2_metadata.tsv \
--p-where "[sorted-fraction] IN ('Neg-Ctrl', 'Pos-Ctrl')" \
--p-exclude-ids True \
--o-filtered-table nc-filtered-table.qza

Saved FeatureTable[Frequency] to: nc-filtered-table.qza
(qiime2-2021.8) 

: 1

In [ ]:
qiime feature-table filter-samples \
--i-table table.qza \
--m-metadata-file tyler_data_metadata.tsv \
--p-where "[source] IN ('mcgill')" \
--o-filtered-table mcgill-table.qza

In [ ]:
qiime feature-table filter-samples \
--i-table table.qza \
--m-metadata-file tyler_data_metadata.tsv \
--p-where "[source] IN ('udm')" \
--o-filtered-table udm-table.qza

In [4]:
qiime feature-table summarize \
    --i-table nc-filtered-table.qza \
    --o-visualization nc-filtered-table.qzv \
    --m-sample-metadata-file DSS2_metadata.tsv

Saved Visualization to: nc-filtered-table.qzv
(qiime2-2021.8) 

: 1

In [5]:
qiime tools view nc-filtered-table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 

### No sequencing controls AND no sheath fluid

In [3]:
qiime feature-table filter-samples \
--i-table table.qza \
--m-metadata-file DSS2_metadata.tsv \
--p-where "[sorted-fraction] IN ('Neg-Ctrl', 'Pos-Ctrl', 'Sheath')" \
--p-exclude-ids True \
--o-filtered-table ncns-filtered-table.qza

Saved FeatureTable[Frequency] to: ncns-filtered-table.qza
(qiime2-2021.8) 

: 1

In [6]:
qiime feature-table summarize \
    --i-table ncns-filtered-table.qza \
    --o-visualization ncns-filtered-table.qzv \
    --m-sample-metadata-file DSS2_metadata.tsv

Saved Visualization to: ncns-filtered-table.qzv
(qiime2-2021.8) 

: 1

In [10]:
qiime tools view ncns-filtered-table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 

#### Exporting the filtered feature table

In [11]:
qiime tools export \
  --input-path ncns-filtered-table.qza \
  --output-path exported-ncns-filtered-table

# feature tables are exported as BIOM v2.1.0 formatted files (BIOMV210DirFmt, .biom)

Exported ncns-filtered-table.qza as BIOMV210DirFmt to directory exported-ncns-filtered-table
(qiime2-2021.8) 

: 1

### No sequencing controls, no sheath fluid, and no contamination (triple n)

In [4]:
# only keeping the 'real' ASVs, as determined by those ASVs which had no or low # of reads in the sheath fluid

qiime taxa filter-table \
--i-table ncns-filtered-table.qza \
--i-taxonomy taxonomy.qza \
--p-exclude Pseudomonadaceae,Rhizobiaceae,Caulobacteraceae,Beijerinckiaceae,Chloroplast,Xanthomonadaceae,Comamonadaceae,Staphylococcaceae,Microbacteriaceae,Xanthobacteraceae,Sphingomonadaceae,Chitinophagaceae,Nitrococcaceae,Sutterellaceae,Solimonadaceae,Alcaligenaceae,Chloroflexi,Nocardiaceae,Burkholderiaceae,Moraxellaceae,Reyranellaceae,Acidobacteriota,Streptococcaceae,Listeriaceae,Mitochondria,Steroidobacteraceae,Sphingobacteriaceae,Nitrosomonadaceae,Acetobacteraceae,Obscuribacteraceae,Rhodobacteraceae \
--o-filtered-table nnn-filtered-table.qza

(qiime2-2021.11) (qiime2-2021.11) Saved FeatureTable[Frequency] to: nnn-filtered-table.qza
(qiime2-2021.11) 

: 1

In [5]:
qiime feature-table summarize \
    --i-table nnn-filtered-table.qza \
    --o-visualization nnn-filtered-table.qzv \
    --m-sample-metadata-file DSS2_metadata.tsv \

#creating a visual summary of the data
#inputting the metadata file here - associating a time point, treatment group to a sample

Saved Visualization to: nnn-filtered-table.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [6]:
qiime tools view nnn-filtered-table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

# Representative Sequences

In [8]:
qiime feature-table tabulate-seqs \
    --i-data rep-seqs.qza \
    --o-visualization rep-seqs.qzv

Saved Visualization to: rep-seqs.qzv
(qiime2-2021.8) 

: 1

In [9]:
qiime tools view rep-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 

# Making A Tree and Performing Diversity Analyses #

## For nnn filtered table (no controls, no sheath, no contaminants)

In [14]:
qiime diversity core-metrics-phylogenetic \
--i-phylogeny rooted-tree.qza \
--i-table nnn-filtered-table.qza \
--p-sampling-depth 1426 \
--m-metadata-file DSS2_metadata.tsv \
  --o-rarefied-table nnn-rarefied-table.qza \
  --o-faith-pd-vector nnn-faith-pd.qza \
  --o-observed-features-vector nnn-observed-otus.qza \
  --o-shannon-vector nnn-shannon-vector.qza \
  --o-evenness-vector nnn-evenness-vector.qza \
  --o-unweighted-unifrac-distance-matrix nnn-unweighted-unifrac-distance-matrix.qza \
  --o-weighted-unifrac-distance-matrix nnn-weighted-unifrac-distance-matrix.qza \
  --o-jaccard-distance-matrix nnn-jaccard-distance-matrix.qza \
  --o-bray-curtis-distance-matrix nnn-bray-curtis-distance-matrix.qza \
  --o-unweighted-unifrac-pcoa-results nnn-unweighted-unifrac-pca-results.qza \
  --o-weighted-unifrac-pcoa-results nnn-weighted-unifrac-pcoa-results.qza \
  --o-jaccard-pcoa-results nnn-jaccard-pcoa-results.qza \
  --o-bray-curtis-pcoa-results nnn-bray-curtis-pcoa-results.qza \
  --o-unweighted-unifrac-emperor nnn-unweighted-unifrac-emperor.qzv \
  --o-weighted-unifrac-emperor nnn-weighted-unifrac-emperor.qzv \
  --o-jaccard-emperor nnn-jaccard-emperor.qzv \
  --o-bray-curtis-emperor nnn-bray-curtis-emperor.qzv

Saved FeatureTable[Frequency] to: nnn-rarefied-table.qza
Saved SampleData[AlphaDiversity] to: nnn-faith-pd.qza
Saved SampleData[AlphaDiversity] to: nnn-observed-otus.qza
Saved SampleData[AlphaDiversity] to: nnn-shannon-vector.qza
Saved SampleData[AlphaDiversity] to: nnn-evenness-vector.qza
Saved DistanceMatrix to: nnn-unweighted-unifrac-distance-matrix.qza
Saved DistanceMatrix to: nnn-weighted-unifrac-distance-matrix.qza
Saved DistanceMatrix to: nnn-jaccard-distance-matrix.qza
Saved DistanceMatrix to: nnn-bray-curtis-distance-matrix.qza
Saved PCoAResults to: nnn-unweighted-unifrac-pca-results.qza
Saved PCoAResults to: nnn-weighted-unifrac-pcoa-results.qza
Saved PCoAResults to: nnn-jaccard-pcoa-results.qza
Saved PCoAResults to: nnn-bray-curtis-pcoa-results.qza
Saved Visualization to: nnn-unweighted-unifrac-emperor.qzv
Saved Visualization to: nnn-weighted-unifrac-emperor.qzv
Saved Visualization to: nnn-jaccard-emperor.qzv
Saved Visualization to: nnn-bray-curtis-emperor.qzv
(qiime2-2021.1

: 1

# Viewing Beta Diversity Metrics (nnn-filtered data)

In [10]:
qiime tools view nnn-weighted-unifrac-emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

In [9]:
qiime tools view nnn-unweighted-unifrac-emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

In [ ]:
qiime tools view nnn-jaccard-emperor.qzv

In [ ]:
qiime tools view nnn-bray-curtis-emperor.qzv

# Beta Diversity Significance Analyses (nnn-filtered data)

## Weighted Unifrac

In [9]:
### beta-group significance on filtered table vectors ####

qiime diversity beta-group-significance \
--i-distance-matrix  nnn-weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sex \
--o-visualization nnn-wunifrac-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

(qiime2-2021.4) (qiime2-2021.4) Saved Visualization to: noout-wunifrac-treat-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-wunifrac-sex-beta-sig.qzv

In [10]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sorted-fraction \
--o-visualization nnn-wunifrac-sf-beta-sig.qzv \
--p-pairwise

# comparing between sorted fractions (EdU+, EdU-, W)

Saved Visualization to: noout-wunifrac-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-wunifrac-sf-beta-sig.qzv

In [10]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column cage \
--o-visualization nnn-wunifrac-cage-beta-sig.qzv \
--p-pairwise

# comparing between sorted fractions (EdU+, EdU-, W)

Saved Visualization to: noout-wunifrac-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-wunifrac-cage-beta-sig.qzv

In [10]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column health-state \
--o-visualization nnn-wunifrac-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states (Pre-DSS, DSS, Recovery, Follow-up)

Saved Visualization to: noout-wunifrac-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-wunifrac-hs-beta-sig.qzv

In [5]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column day \
--o-visualization nnn-wunifrac-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: nnn-wunifrac-day-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [6]:
qiime tools view nnn-wunifrac-day-beta-sig.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

## Unweighted Unifrac

In [11]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sex \
--o-visualization nnn-un-unifrac-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

Saved Visualization to: noout-un-unifrac-treat-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-un-unifrac-sex-beta-sig.qzv

In [12]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sorted-fraction \
--o-visualization nnn-un-unifrac-sf-beta-sig.qzv \
--p-pairwise

# comparing between sorted fractions

Saved Visualization to: noout-un-unifrac-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-un-unifrac-sf-beta-sig.qzv

In [12]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column cage \
--o-visualization nnn-un-unifrac-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: noout-un-unifrac-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-un-unifrac-cage-beta-sig.qzv

In [12]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column health-state \
--o-visualization nnn-un-unifrac-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states

Saved Visualization to: noout-un-unifrac-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-un-unifrac-hs-beta-sig.qzv

In [7]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column day \
--o-visualization nnn-un-unifrac-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: nnn-un-unifrac-day-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [8]:
qiime tools view nnn-un-unifrac-day-beta-sig.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

## Jaccard

In [13]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-jaccard-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sex \
--o-visualization nnn-jaccard-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

Saved Visualization to: noout-jaccard-treat-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-jaccard-sex-beta-sig.qzv

In [14]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-jaccard-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sorted-fraction \
--o-visualization nnn-jaccard-sf-beta-sig.qzv \
--p-pairwise

# comparing between sorted fractions

Saved Visualization to: noout-jaccard-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-jaccard-sf-beta-sig.qzv

In [14]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-jaccard-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column cage \
--o-visualization nnn-jaccard-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: noout-jaccard-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-jaccard-cage-beta-sig.qzv

In [14]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-jaccard-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column health-state \
--o-visualization nnn-jaccard-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states

Saved Visualization to: noout-jaccard-sf-beta-sig.qzv
(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) 

: 1

In [ ]:
qiime tools view nnn-jaccard-hs-beta-sig.qzv

In [9]:
qiime diversity beta-group-significance \
--i-distance-matrix  nnn-jaccard-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column day \
--o-visualization nnn-jaccard-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: nnn-jaccard-day-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [10]:
qiime tools view nnn-jaccard-day-beta-sig.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

## Bray-Curtis

In [35]:
qiime diversity beta-group-significance \
--i-distance-matrix nnn-bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sex \
--o-visualization nnn-bc-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

Saved Visualization to: nnn-bc-sex-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [ ]:
qiime tools view nnn-bc-sex-beta-sig.qzv

In [36]:
qiime diversity beta-group-significance \
--i-distance-matrix nnn-bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column sorted-fraction \
--o-visualization nnn-bc-sf-beta-sig.qzv \
--p-pairwise

# comparing between sorted fractions

Saved Visualization to: nnn-bc-sf-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [ ]:
qiime tools view nnn-bc-sf-beta-sig.qzv

In [37]:
qiime diversity beta-group-significance \
--i-distance-matrix nnn-bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column cage \
--o-visualization nnn-bc-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: nnn-bc-cage-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [ ]:
qiime tools view nnn-bc-cage-beta-sig.qzv

In [38]:
qiime diversity beta-group-significance \
--i-distance-matrix nnn-bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column health-state \
--o-visualization nnn-bc-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states

Saved Visualization to: nnn-bc-hs-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [ ]:
qiime tools view nnn-bc-hs-beta-sig.qzv

In [11]:
qiime diversity beta-group-significance \
--i-distance-matrix nnn-bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS2_metadata.tsv \
--m-metadata-column day \
--o-visualization nnn-bc-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: nnn-bc-day-beta-sig.qzv
(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) 

: 1

In [12]:
qiime tools view nnn-bc-day-beta-sig.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

# Alpha diversity significance analyses (nnn-filtered data)

In [60]:
qiime diversity alpha-group-significance \
--i-alpha-diversity nnn-shannon-vector.qza \
--m-metadata-file DSS2_metadata.tsv \
--o-visualization nnn-shannon-alpha-diversity.qzv 

Saved Visualization to: nnn-shannon-alpha-diversity.qzv
(qiime2-2021.11) 

: 1

In [61]:
qiime tools view nnn-shannon-alpha-diversity.qzv 

# no sig diffs btwn any parameters

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

# Classifier & Taxonomy #

In [7]:
# trains a classifier on the Silva database for V4-5 region (515F, 926R)

#Extract reference reads:
qiime feature-classifier extract-reads \
--i-sequences silva-138-99-seqs.qza \
--p-f-primer GTGYCAGCMGCCGCGGTAA \
--p-r-primer CCGYCAATTYMTTTRAGTTT \
--p-trunc-len 0 \
--p-min-length 100 \
--p-max-length 450 \
--o-reads ref-seqs-v138-V4V5.qza

(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) Saved FeatureData[Sequence] to: ref-seqs-v138-V4V5.qza
(qiime2-2021.4) 

: 1

In [9]:
#Train the classifier (needs about 20 GB memory from my experience):

qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ref-seqs-v138-V4V5.qza \
--i-reference-taxonomy silva-138-99-tax.qza \
--o-classifier silva-138-V4V5-classifier.qza

(qiime2-2021.4) Killed
(qiime2-2021.4) 

: 1

In [5]:
#Get the taxonomy of your sequences using the trained classifier (needs about 6-10 Gbs using 8 threads)

qiime feature-classifier classify-sklearn \
--i-classifier silva-138-V4V5-classifier.qza \
--i-reads rep-seqs.qza \
--o-classification taxonomy.qza \
--p-n-jobs 8

#--verbose \
#--p-reads-per-batch 1000

Killed
(qiime2-2020.8) 

: 1

In [3]:
# The taxonomy.qza file relates ASVs to taxonomic classification & gives % confidence of classifications
# we generate a visualization of this file as below

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

(qiime2-2021.11) (qiime2-2021.11) (qiime2-2021.11) Saved Visualization to: taxonomy.qzv
(qiime2-2021.11) 

: 1

In [4]:
qiime tools view taxonomy.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

In [20]:
qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file DSS2_metadata.tsv \
--o-visualization taxa-bar-plots.qzv

#using the qiime taxa bar plots command to generate bar plots with various taxonomy levels 
#can sort by metadata but visualization isnt great with so many samples 
#grouped bar plots and longitudinal - if time series data - can help with this 

Saved Visualization to: taxa-bar-plots.qzv
(qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) (qiime2-2021.8) 

: 1

In [21]:
qiime tools view taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.8) 